<a href="https://colab.research.google.com/github/mofuss/gee2mofuss/blob/main/scripts/GEE2MoFuSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>
  <img src="https://www.mofuss.unam.mx/mofuss/img/logo-white_1.png" width="180">
  <img src="https://www.fao.org/images/corporatelibraries/fao-logo/fao-logo-archive/fao-logo-blue-3lines-en.svg" width="180">
</p>





# 1. Initialize authentication and initialization of Google Earth Engine



In [ ]:
import ee
import geemap
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# Initialize authentication and initialization of Google Earth Engine
ee.Authenticate()
ee.Initialize(project='gee2mofuss')

print("\033[92m✅ Authentication and Initialization successful\033[0m")

✅ Authentication and Initialization successful


# 2. Area of Interest (AoI)

## 2.1 Displays available Area of Interest (AoIs) for Region and Country

In this section, the names of the available "countries" and "mofuss_regions" are extracted.

In [ ]:
LSIBs = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
mofuss_regions0 = ee.FeatureCollection("users/aghilardi/mofuss_regions0_simp")

# Pull names/codes
names = mofuss_regions0.aggregate_array('NAME_0').getInfo()
codes = mofuss_regions0.aggregate_array('GID_0').getInfo()

# Hard fixes (exact strings seen in your output)
FIXUPS = {
    "M\u00e9xico": "México",  # just in case (correct form stays same)
    "M�xico": "México",
    "C�te d'Ivoire": "Côte d’Ivoire",
    "Cote d'Ivoire": "Côte d’Ivoire",
    "S�o Tom� and Pr�ncipe": "São Tomé and Príncipe",
    "Sao Tome and Principe": "São Tomé and Príncipe",
}

fixed_names = [FIXUPS.get(n, n) for n in names]
combined_countries = sorted(zip(fixed_names, codes), key=lambda x: x[0])

# Pretty print (adjust columns if you like)
print("📍 Available countries:")
cols_per_line = 3
max_len = max((len(n) for n, _ in combined_countries), default=0)

for i in range(0, len(combined_countries), cols_per_line):
    chunk = combined_countries[i:i+cols_per_line]
    line = "   ".join([f"{name:<{max_len}} {code:<3}" for name, code in chunk])
    print(line)


# Available regions
print("📍 Available regions:")

# Get and sort unique regions
regions = mofuss_regions0.aggregate_array('mofuss_reg').distinct().sort().getInfo()

# Column settings
cols_per_line = 3
max_len = max((len(r) for r in regions), default=0)

# Print in neat columns
for i in range(0, len(regions), cols_per_line):
    chunk = regions[i:i+cols_per_line]
    line = "   ".join([f"{r:<{max_len}}" for r in chunk])
    print(line)



📍 Available countries:
Afghanistan                      AFG   Algeria                          DZA   Angola                           AGO
Argentina                        ARG   Armenia                          ARM   Azerbaijan                       AZE
Bangladesh                       BGD   Benin                            BEN   Bhutan                           BTN
Bolivia                          BOL   Botswana                         BWA   Brazil                           BRA
Burkina Faso                     BFA   Burundi                          BDI   Cabo Verde                       CPV
Cambodia                         KHM   Cameroon                         CMR   Central African Republic         CAF
Chad                             TCD   China                            CHN   Colombia                         COL
Comoros                          COM   Costa Rica                       CRI   Côte d’Ivoire                    CIV
Democratic Republic of the Congo COD   Djibouti          

## 2.2 Draw a user-defined-AoI (optional)



In [ ]:
Map = geemap.Map(center=[0, 0], zoom=6)
Map.add_basemap('SATELLITE')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Get the geometry the user drew with the Draw control in geemap
# (works for Polygon/Rectangle; fallbacks for Feature/FC)

aoi = getattr(Map, "user_roi", None) or getattr(Map, "user_roi_bounds", None)

# Fallback: try the last drawn feature
if aoi is None:
    last = getattr(Map, "draw_last_feature", None)
    if last is not None:
        aoi = ee.Feature(last).geometry()

def _to_geometry(obj):
    """Coerce obj to ee.Geometry if possible; else return None."""
    try:
        if obj is None:
            return None
        # Already a geometry?
        if isinstance(obj, ee.geometry.Geometry):
            return obj
        # Feature -> geometry
        if isinstance(obj, ee.feature.Feature):
            return obj.geometry()
        # FeatureCollection -> union geometry
        if isinstance(obj, ee.featurecollection.FeatureCollection):
            return obj.geometry()
        # Some geemap draw tools hand back plain GeoJSON dicts
        if isinstance(obj, dict) and obj.get("type"):
            return ee.Geometry(obj)
        # Last resort: hope it can be wrapped as geometry
        return ee.Geometry(obj)
    except Exception:
        return None

d_aoi = _to_geometry(aoi)

if d_aoi is not None:
    print("\033[92m✅ User-defined AOI successfully retrieved\033[0m")
    # Optional visualization
    # Map.addLayer(d_aoi, {}, "Drawn AOI")
    # Map.centerObject(d_aoi, 10)
else:
    print("\033[93m⚠️ No user-defined AoI found. Proceeding without an AoI (expecting a predefined AoI later).\033[0m")

# Tip for later cells:
# if d_aoi is None:
#     # skip AOI-dependent steps or use a predefined geometry
#     ...

✅ User-defined AOI successfully retrieved


## 2.3 Input an AoI

In this section, xxx

In [ ]:
# --- Config (strings only) ---
ty_aoi = 'user-defined'                 # 'world' | 'regions' | 'countries' | 'user-defined'
country_iso3 = 'ZMB'                 # e.g., 'ZMB'
region_code  = 'SSA_adm0_zambia'     # e.g., 'SSA_adm0_zambia'

# --- FeatureCollections you already have in your notebook ---
# Use the same one if it contains both 'mofuss_reg' and 'GID_0'
regions_fc   = mofuss_regions0
countries_fc = mofuss_regions0

def select_aoi(ty_aoi, region_id=None, *, regions_fc=None, countries_fc=None, dynamic_geom=None):
    if ty_aoi == 'world':
        geom = ee.Geometry.Rectangle([-180, -89.9, 180, 89.9], 'EPSG:4326', False)
        zoom = 1

    elif ty_aoi == 'regions':
        feat = regions_fc.filter(ee.Filter.eq('mofuss_reg', region_id)).union()
        geom = ee.Feature(feat).geometry()
        zoom = 3

    elif ty_aoi == 'countries':
        feat = countries_fc.filter(ee.Filter.eq('GID_0', region_id)).first()
        if feat is None:
            raise ValueError(f"No country matched GID_0 == {region_id}")
        geom = ee.Feature(feat).geometry()
        zoom = 6

    elif ty_aoi == 'user-defined':
        if dynamic_geom is None:
            raise ValueError("Provide dynamic_geom when ty_aoi='user-defined'.")
        geom = dynamic_geom
        zoom = 12

    else:
        raise ValueError("ty_aoi must be 'world', 'regions', 'countries', or 'user-defined'.")

    return geom, zoom

# --- Decide region_id from ty_aoi (strings only) ---
if ty_aoi == 'countries':
    region_id = country_iso3
elif ty_aoi == 'regions':
    region_id = region_code
else:
    region_id = None

# --- Get AOI ---
aoi, zoom = select_aoi(
    ty_aoi,
    region_id,
    regions_fc=regions_fc,
    countries_fc=countries_fc,
    dynamic_geom=(d_aoi if ty_aoi == 'user-defined' else None)
)

print(f"✅ AOI selected for '{ty_aoi}': {region_id if region_id else 'World'}, Zoom: {zoom}")

# If you're using geemap/Map in Colab:
# Map.centerObject(aoi, zoom)
# Map.addLayer(aoi, {}, f"AOI: {ty_aoi} - {region_id if region_id else 'World'}")


✅ AOI selected for 'user-defined': World, Zoom: 12


## 2.4 Visualize input AoI

In [ ]:
Map = geemap.Map(center=[0, 0], zoom=6)
Map.add_basemap('SATELLITE')
Map.centerObject(aoi)

Map.addLayer(aoi,{}, "aoi")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# 3. Download MoFuSS input layers
Note that several layers are downloaded outside of Google Earth Engine, using the available R scripts. See full MoFuSS documentation (add link).

## 3.1 Land Use / Cover
Image collections for Land Use / Cover products are processed by applying time filters and masks.

Explore products here:
1. [MODIS](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1)
2. [Copernicus](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global)
3. [Dynamic World](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1)







In [ ]:
#Land Cover periods
mod_period = ('2010-01-01', '2010-12-31')
cop_period = ('2015-01-01', '2015-12-31')
dw_period  = ('2016-01-01', '2016-12-31')

copernicusBands = ['discrete_classification', 'discrete_classification-proba']

modis_lc = ee.ImageCollection("MODIS/061/MCD12Q1")
copernicus_lc = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
DW = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

modisLc = modis_lc.select('LC_Type1').filterDate(*mod_period).first().selfMask()
copernicusLc = copernicus_lc.select(copernicusBands).filterDate(*cop_period).first().selfMask()
dynamic_world = DW.select("label").filterDate(*dw_period).mode()

print("\033[92m✅ Land Use / Cover products processed successfully\033[0m")

✅ Land Use / Cover products processed successfully


## 3.2 Global Forest Change and GFCC

The loss, gain, and loss year bands are selected for the Global Forest Change (GFC) dataset, and the median is calculated for different periods of the GFCC.


In [ ]:
GFC = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

loss = GFC.select('loss').selfMask()
yearLoss = GFC.select('lossyear').selfMask()
gain = GFC.select('gain').selfMask()

GFCC = ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3")
GFCC2000 = GFCC.filterDate('2000-01-01','2001-01-01').median().select(['tree_canopy_cover','uncertainty']).selfMask().float()
GFCC2005 = GFCC.filterDate('2005-01-01','2006-01-01').median().select(['tree_canopy_cover','uncertainty']).selfMask().float()
GFCC2010 = GFCC.filterDate('2010-01-01','2011-01-01').median().select(['tree_canopy_cover','uncertainty']).selfMask().float()
GFCC2015 = GFCC.filterDate('2015-01-01','2016-01-01').median().select(['tree_canopy_cover','uncertainty']).selfMask().float()

print("\033[92m✅ Global Forest Change products processed successfully\033[0m")

✅ Global Forest Change products processed successfully


## 3.3 Elevation



In [ ]:
SRTM = ee.Image("CGIAR/SRTM90_V4")

JRC_GSW = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
GLCG_IW = ee.ImageCollection("GLCF/GLS_WATER")
WWF_rivers = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers")
JRC_Historical = ee.ImageCollection("JRC/GSW1_4/YearlyHistory")

JRC_GSWPro = JRC_GSW.selfMask().int()
GLCG_IWPro = GLCG_IW.map(lambda image: image.updateMask(image.select('water').eq(2))).median()
JRC_HistoricalPro = JRC_Historical.mode().int()

print("\033[92m✅ Elevation products processed successfully\033[0m")

✅ Elevation products processed successfully


## 3.4 Aboveground Biomass
The biomass and carbon datasets, such as WCMC, WHRC, GEDI, DAAC, and ESA_AGB_CCI, are loaded.
Here is an example of how the visualization would be prepared (in Python, the data is usually exported or processed for further analysis).

In [ ]:
# WHRC = ee.Image("WHRC/biomass/tropical")
# GEDI = ee.Image("LARSE/GEDI/GEDI04_B_002")

DAAC = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")
ESA_AGB_CCI = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
WCMC = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0")

daac = DAAC.select("agb").first().clip(aoi)
esa_agb_2010 = ESA_AGB_CCI.filterDate('2009-01-01','2011-01-01').select("AGB").median()
esa_agb_2020 = ESA_AGB_CCI.filterDate('2019-01-01','2021-01-01').select("AGB").median()

wcmc = WCMC.select("carbon_tonnes_per_ha").median()
wcmc_biomass = wcmc.divide(0.47).rename("biomass_tonnes_per_ha")

print("\033[92m✅ Aboveground Biomass products processed successfully\033[0m")

✅ Aboveground Biomass products processed successfully


# 4. Visualization (*in progess*)

In [ ]:
import geemap

Map = geemap.Map(center=[0, 0], zoom=6)
Map.add_basemap('SATELLITE')
Map.centerObject(aoi)

# Add the MODIS Land Cover dataset with a color palette (adjust visualization parameters as needed)
vis_params = {
    'min': 1,
    'max': 17,
    'palette': [
        '#05450a','#086a10','#54a708','#78d203',
        '#009900','#c6b044','#dcd159','#dade48',
        '#fbff13','#b6ff05','#27ff87','#c24f44',
        '#a5a5a5','#ff6d4c','#69fff8','#f9ffa4',
        '#1c0dff'
    ]
}

# Choose the dataset you want to display
Map.addLayer(daac.clip(aoi), {}, "dw")
Map.addLayer(aoi, {}, "AoI")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# 5. Export to Google Drive folder
In this section, a function is defined to export images to Google Drive using ee.batch.Export.image.toDrive.
Calls to the function are made for each processed dataset.

## 5.1 Download layers for a single AoI



In [ ]:
import time
import re

# -------- EXPORT --------
def export_image_to_drive(image, description, scale, crs, folder, region):
    params = {
        'image': image,
        'description': description,
        'folder': folder,
        'scale': scale,
        'crs': crs,
        'region': region,
        'maxPixels': 1e13
    }
    task = ee.batch.Export.image.toDrive(**params)
    task.start()
    print("🚀 Export started:", description)
    return task

def sanitize_name(name, max_len=80):
    clean = re.sub(r'[^A-Za-z0-9\.\,;:_-]', '_', name)
    clean = re.sub(r'_+', '_', clean)
    return clean[:max_len]

def batch_export_single_aoi(datasets, fixed_scale, crs, aoi_name, aoi_geom, folder, res_mode="off"):
    tasks = []
    for idx, img in enumerate(datasets):
        scale = fixed_scale if (ty_aoi == 'world' or res_mode == "on") else img.projection().nominalScale().getInfo()
        try:
            raw_id = img.get('system:id').getInfo()
            dataset_id = sanitize_name(raw_id)
        except:
            raw_band = img.bandNames().get(0).getInfo()
            dataset_id = sanitize_name(raw_band)
        label = f"{ty_aoi}_{dataset_id}"
        desc = f"{label}_{aoi_name}_{int(scale)}m"
        img_to_export = img.clip(aoi_geom)
        task = export_image_to_drive(
            image=img_to_export,
            description=desc,
            scale=scale,
            crs=crs,
            folder=folder,
            region=aoi_geom
        )
        tasks.append(task)

    while True:
        print("\n📊 Export status:")
        done = True
        for t in tasks:
            st = t.status()
            print(f"• {st['description']}: {st['state']}")
            if st['state'] not in ('COMPLETED', 'FAILED', 'CANCELLED'):
                done = False
        if done:
            print("\033[92m✅ All exports finished successfully\033[0m")
            break
        time.sleep(120)

In [ ]:
# Export parameters
epsg = 'EPSG:4326'
folder = 'GEE2MoFuSS' # Your Google Drive folder, no path is needed. If more than one folder with the same name exists, GEE will choose the first one.
res_mode = "on" # Options 'on' or 'off'
fixed_scale = 1000  # Fixed scale if res_mode = 'on'

aoi_simple = {region_id: aoi}
datasets =[GFCC2000, gain, yearLoss, SRTM, modisLc, copernicusLc, dynamic_world, daac, wcmc_biomass, esa_agb_2020]
# datasets =[SRTM]

# ------------------ Call main function ------------------
batch_export_single_aoi(
    datasets=datasets,
    fixed_scale=fixed_scale,
    crs=epsg,
    aoi_name=region_id,
    aoi_geom=aoi,
    folder=folder,
    res_mode=res_mode
)

🚀 Export started: user-defined_tree_canopy_cover_None_1000m
🚀 Export started: user-defined_UMD_hansen_global_forest_change_2021_v1_9_None_1000m
🚀 Export started: user-defined_UMD_hansen_global_forest_change_2021_v1_9_None_1000m
🚀 Export started: user-defined_CGIAR_SRTM90_V4_None_1000m
🚀 Export started: user-defined_MODIS_061_MCD12Q1_2010_01_01_None_1000m
🚀 Export started: user-defined_COPERNICUS_Landcover_100m_Proba-V-C3_Global_2015_None_1000m
🚀 Export started: user-defined_label_None_1000m
🚀 Export started: user-defined_NASA_ORNL_biomass_carbon_density_v1_2010_None_1000m
🚀 Export started: user-defined_biomass_tonnes_per_ha_None_1000m
🚀 Export started: user-defined_AGB_None_1000m

📊 Export status:
• user-defined_tree_canopy_cover_None_1000m: READY
• user-defined_UMD_hansen_global_forest_change_2021_v1_9_None_1000m: READY
• user-defined_UMD_hansen_global_forest_change_2021_v1_9_None_1000m: READY
• user-defined_CGIAR_SRTM90_V4_None_1000m: READY
• user-defined_MODIS_061_MCD12Q1_2010_01_01

## 5.2 Download layers for multiple AoIs simultaneously (*in progress*)


In [ ]:
# import os
# from googleapiclient.discovery import build
# from google.colab import drive

# def create_drive_folders_colab(main_folder_name, aoi_names):
#     # Montar Google Drive en Colab
#     drive.mount('/content/drive')

#     # Crear carpeta principal
#     base_path = f"/content/drive/MyDrive/{main_folder_name}"
#     os.makedirs(base_path, exist_ok=True)

#     # Crear subcarpetas por AoI
#     aoi_folder_paths = {}
#     for aoi_name in aoi_names:
#         aoi_path = os.path.join(base_path, aoi_name)
#         os.makedirs(aoi_path, exist_ok=True)
#         aoi_folder_paths[aoi_name] = aoi_path

#     return aoi_folder_paths


# aois_multiples = aois_dict_random


# aoi_folder_paths = create_drive_folders_colab(folder, list(aois_multiples.keys()))
# aoi_folder_paths

In [ ]:
# import time
# import os

# def export_image_to_drive(image, description, scale, crs, aoiExp, folder):
#     task = ee.batch.Export.image.toDrive(
#         image=image,
#         description=description,
#         folder=folder,
#         scale=scale,
#         region=aoiExp,
#         crs=crs,
#         maxPixels=1e12
#     )
#     task.start()
#     print("Export iniciado para:", description)


# def batch_export(datasets, fixed_scale, crs, aois, aoi_folder_paths, res_mode="off"):
#     tasks = []

#     for aoi_name, aoi_geom in aois.items():
#         folder_name = os.path.basename(aoi_folder_paths.get(aoi_name, ""))

#         for img in datasets:
#             band_name = img.bandNames().get(0).getInfo()
#             scale = fixed_scale if res_mode == "on" else img.projection().nominalScale().getInfo()

#             desc = f"{band_name}_{aoi_name}_{int(scale)}scale"
#             export_image_to_drive(img.clip(aoi_geom), desc, scale, crs, aoi_geom, folder_name)
#             tasks.append(desc)

#     # Monitorear tareas
#     while True:
#         print("\nEstado de exportaciones:")
#         completed = True
#         task_list = ee.batch.Task.list()
#         for task in task_list[:len(tasks)]:
#             task_status = task.status()['state']
#             task_desc = task.status()['description']
#             print(f"{task_desc}: {task_status}")
#             if task_status not in ['COMPLETED', 'FAILED', 'CANCELLED']:
#                 completed = False
#         if completed:
#             print("Todas las exportaciones han finalizado.")
#             break
#         time.sleep(30)

In [ ]:
# res_mode = "on"
# fixed_scale = 1000  # escala fija si res_mode="on"

# datasets = [di, SRTM]  # Lista de productos

# aois_multiples = aois_dict_random  # Diccionario de múltiples AoIs previamente definido

# # Ejecutar exportación
# batch_export(
#     datasets=datasets,
#     fixed_scale=fixed_scale,
#     crs=epsg,
#     aois=aois_multiples,
#     aoi_folder_paths=aoi_folder_paths,
#     res_mode=res_mode
# )